# Code for the Paper: Managing inventories for perishable e-groceries: The value of probabilistic information

This notebook accompanies the paper "Managing inventories for perishable e-groceries: The value of probabilistic information". It contains a simulation an inventory management system with perishable goods. It models demand, supply, and spoilage uncertainties and optimizes order quantities to minimize costs.

### Imports
We import necessary libraries including `numpy` for numerical operations, `numba` for JIT compilation (speeding up loops), and `scipy` for statistical distributions and optimization.

In [2]:
#export
import numpy as np
import quantecon as qe

import matplotlib.pyplot as plt

from numba import vectorize
from numba import jit, njit, prange

from numba import int32, float32, double, boolean    # import the types
from numba.experimental import jitclass

from scipy import stats
from scipy.stats import poisson
from scipy.stats import nbinom
from scipy.optimize import minimize

import time
import math
import csv

### Helper Functions
These functions assist with random number generation and seeding, ensuring reproducibility in Numba-compiled code.

In [3]:
#export

@njit
def rand_choice_nb(arr, prob):
    """
    :param arr: A 1D numpy array of values to sample from.
    :param prob: A 1D numpy array of probabilities for the given samples.
    :return: A random sample from the given array with a given probability.
    """
    return arr[np.searchsorted(np.cumsum(prob), np.random.random(), side="right")]

In [4]:
#export

@njit
def set_numba_numpy_random_seed(seed):
    np.random.seed(seed)

def set_numpy_random_seed(seed):
    np.random.seed(seed)
    set_numba_numpy_random_seed(seed)
    

### Statistical Utilities
Functions for generating random variables from specific distributions.

In [5]:
#export

@njit
def inverse_transform_binomial(n,p,u):    
    c= p/(1-p)    
    #print (c)
    i = 0
    pr = (1-p)**n    
    F = pr
    
    while i < n+1:        
        if u <= F:
            return i          
        pr = ( (c*(n-i)) / (i+1) ) * pr        
        F = F+pr    
        i+=1            
    return i-1

### Data Generation
These functions generate the stochastic data for the simulation:
- **Demand**: Modeled using Poisson and Negative Binomial distributions.
- **Supply**: Modeled with a transition matrix for supply states and Beta distribution for shortage fractions.
- **Spoilage**: Modeled with uniform probabilities.

In [6]:
#export

#@njit
def generate_demand_data(number_of_simulation_periods, number_of_optimisation_paths, lambda_mu, lambda_omega, seed, demand_is_deterministic=False):
        
    demand_evaluation = np.full(number_of_simulation_periods, 0)
    demand_optimisation = np.full((number_of_optimisation_paths, number_of_simulation_periods), 0)

    for i in range(number_of_simulation_periods):
        set_numpy_random_seed(seed*(i+1))
        mu =  np.random.poisson(lambda_mu)
        omega = np.random.poisson(lambda_omega)
        sigma = np.sqrt(mu + omega)

        r0 = mu*mu / (sigma*sigma - mu)
        p = (sigma*sigma - mu) / (sigma*sigma)
    
        set_numpy_random_seed(seed*(i+1))
        demand_evaluation[i] = np.random.negative_binomial(r0, 1-p)
    
        if demand_is_deterministic == False:
            set_numpy_random_seed(seed*i+1000)        
            demand_optimisation[:,i] = np.random.negative_binomial(r0, 1-p, number_of_optimisation_paths)
        
        else:
            demand_optimisation[:,i] = np.full(number_of_optimisation_paths, mu)

    return demand_optimisation, demand_evaluation

In [7]:
#export

#@njit
def generate_supply_data_evaluation(number_of_simulation_periods, pi, beta1, beta2, type_last_period, seed):

    set_numpy_random_seed(seed)
    fail_type_current = rand_choice_nb(np.arange(3), pi[type_last_period:,])
    if fail_type_current < 2:
        relative_shortage_evaluation = fail_type_current

    else:
        relative_shortage_evaluation = np.random.beta(beta1, beta2)

    for i in range(1, number_of_simulation_periods):
        set_numpy_random_seed(seed+i)
        fail_type_current = rand_choice_nb(np.arange(3), pi[fail_type_current:,])
    
        if fail_type_current < 2:
            relative_shortage_evaluation = np.append(relative_shortage_evaluation, fail_type_current)

        else:
            set_numpy_random_seed(seed+10*i+100)
            relative_shortage_evaluation = np.append(relative_shortage_evaluation, np.random.beta(beta1, beta2))
    
    return relative_shortage_evaluation


In [8]:
#export

@njit
def generate_supply_data_optimisation(pi, pi_stat, beta1, beta2, number_of_optimisation_paths, tau, nu, type_last_period, seed, supply_is_deterministic=False):

    if supply_is_deterministic == False:
        relative_shortage_optimisation = np.full((number_of_optimisation_paths, tau+nu+1), 0, dtype = np.float32)
        
        for i in range(number_of_optimisation_paths):
            set_numba_numpy_random_seed(seed+i)
            
            fail_type_current = rand_choice_nb(np.arange(3), pi[type_last_period:,])
            
            relative_shortage = np.empty(tau+nu+1, dtype = np.float32)
        
            if fail_type_current < 2:
                relative_shortage[0] = fail_type_current

            else:
                relative_shortage[0] = np.random.beta(beta1, beta2)
            
            if tau+nu > 0:
                for j in range(tau+nu):
                    fail_type_current = rand_choice_nb(np.arange(3), pi[fail_type_current:,])
    
                    if fail_type_current < 2:
                        relative_shortage[j+1] = fail_type_current

                    else:
                        relative_shortage[j+1] = np.random.beta(beta1, beta2)
                
            relative_shortage_optimisation[i,:] = relative_shortage
                
    else:
        relative_shortage_optimisation = np.full((number_of_optimisation_paths, tau+nu+1), pi_stat[1]+pi_stat[2]*beta1/(beta1+beta2), dtype=np.float32)

    return relative_shortage_optimisation


In [9]:
#export

#@njit
def generate_spoilage_data(spoil_probabilities, number_of_optimisation_paths, number_of_simulation_periods, seed, spoilage_is_deterministic=False):
    
    max_number_of_spoil = len(spoil_probabilities)
    set_numpy_random_seed(seed)
    spoilage_data_evaluation = np.array(np.zeros([max_number_of_spoil, number_of_simulation_periods]))
    spoilage_data_optimisation = np.array(np.zeros([max_number_of_spoil, number_of_optimisation_paths, number_of_simulation_periods]))
    for i in range(max_number_of_spoil):
        set_numpy_random_seed(seed+30*i)
        spoilage_data_evaluation[i,:] = np.random.uniform(0, 1, number_of_simulation_periods)
        if spoilage_is_deterministic == False:
            for j in range(number_of_optimisation_paths):
                set_numpy_random_seed(seed+20*i+10*j+100)
                spoilage_data_optimisation[i,j,:] = np.random.uniform(0, 1, number_of_simulation_periods)
    
    if spoilage_is_deterministic == True:
        spoilage_data_optimisation = np.zeros([max_number_of_spoil, number_of_optimisation_paths, number_of_simulation_periods])
        foo = 0
        for i in range(max_number_of_spoil):
            foo += (i+1)*spoil_probabilities[i]
        
        expec = np.int8(np.around(foo))
            
        spoilage_data_optimisation[expec-1,:,:] = 1
        
    return spoilage_data_optimisation, spoilage_data_evaluation


### Inventory Logic
Core functions defining how inventory changes in each period:
- **add_supply**: Updates inventory with arriving orders.
- **take_out_demand**: Fulfills demand from inventory, calculating underage.
- **get_spoilage**: Removes spoiled items based on age.
- **transfer_inventory**: Ages the inventory by one period.
- **period_transition**: Combines these steps to simulate one full period and calculate costs.

In [10]:
#export

@njit
def add_supply(supply, order):
    if order < 0:
        print ("order error")

    else:
        realised_supply = np.round((1 - supply) * order)

    return realised_supply

In [11]:
#export

@njit
def take_out_demand(inventory, demand):
    total_inventory = np.sum(inventory)
    underage = max(demand - total_inventory, 0)

    if underage > 0:
        inventory.fill(0)

    else:
        remaining_demand = demand
    
        for row in range(inventory.shape[0]-1,-1 ,-1):  
            if remaining_demand >= inventory[row]:    
                remaining_demand -= inventory[row]
                inventory[row] = 0
            elif remaining_demand > 0:
                inventory[row] -= remaining_demand
                if inventory[row] < 0:
                    print ("Something wrong here: Negative Inventory")
                remaining_demand = 0
            
    return inventory, underage
        

In [12]:
#export

@njit
def get_spoilage(spoil_probabilities, inventory, spoilage_evaluation):

    spoilage_random = spoilage_evaluation
    
    max_number_of_spoil = len(spoil_probabilities)
    spoil_probability_period = np.zeros(max_number_of_spoil)
    spoilage = np.zeros(max_number_of_spoil)
    
    for period in range(0, max_number_of_spoil-1):
        spoil_probability_period[period] = spoil_probabilities[period]/(1-np.sum(spoil_probabilities[:period]))
        
    spoil_probability_period[max_number_of_spoil-1] = 1


    for i in range(max_number_of_spoil-1):
        spoilage[i] = inverse_transform_binomial(inventory[i], spoil_probability_period[i], spoilage_random[i])
        inventory[i] -= spoilage[i]

    spoilage[max_number_of_spoil-1] = inventory[max_number_of_spoil-1]
    inventory[max_number_of_spoil-1] = 0

    spoilage_sum = np.sum(spoilage)

    return inventory, spoilage_sum

In [13]:
#export

@njit
def transfer_inventory(inventory):
    inventory_next_period = np.roll(inventory, 1)    
    inventory_next_period[0] = 0
    
    return inventory_next_period

In [14]:
#export

@njit
def period_transition(inventory, supply_evaluation_period, orders_period, demand_evaluation_period, spoil_probabilities, spoilage_evaluation_period, h, v, b):
    cost = 0
    inventory[0] = add_supply(supply_evaluation_period, orders_period)
    inventory, underage = take_out_demand(inventory, demand_evaluation_period)
    cost += underage*b
    inventory, spoilage = get_spoilage(spoil_probabilities, inventory, spoilage_evaluation_period)
    cost += spoilage*h + np.sum(inventory)*v
    inventory = transfer_inventory(inventory)
    
    return inventory, cost, spoilage, underage

### Optimization
Functions to find the optimal order quantity:
- **calculate_starting_inventory**: Simulates the system for the lead time (`tau`) to get the state before decision making.
- **calculate_order_costs**: Evaluates the cost of a specific order decision over the planning horizon (`nu`).
- **optimise_order_decision**: Uses `scipy.optimize.minimize` to find the best order quantity that minimizes expected costs.

In [15]:
@njit
def calculate_starting_inventory(number_of_optimisation_paths, init_inventory, supply_optimisation_path, orders_path, demand_optimisation_path, spoil_probabilities, spoilage_optimisation_path, tau, h, v, b):
    inventory_state = np.full((number_of_optimisation_paths, len(spoil_probabilities)), 0)
    for i in range(number_of_optimisation_paths):
        inventory = init_inventory.copy()
        for j in range(tau):
            inventory, cost, foo1, foo2 = period_transition(inventory, supply_optimisation_path[i,j], orders_path[j], demand_optimisation_path[i,j], spoil_probabilities, spoilage_optimisation_path[:,i,j], h, v, b)
        inventory_state[i,:] = inventory

    return inventory_state

In [16]:
@njit
def calculate_order_costs(theta_star, number_of_optimisation_paths, inventory_start, supply_optimisation_path, demand_optimisation_path, spoil_probabilities, spoilage_optimisation_path, nu, rho, h, v, b):
    path_cost = np.zeros(number_of_optimisation_paths, dtype = np.float32)
    inventory_paths = inventory_start.copy()
    for i in range(number_of_optimisation_paths):
        inventory = inventory_paths[i,:]
        total_cost = 0
        for j in range(nu+1):
            inventory, cost, foo1, foo2 = period_transition(inventory, supply_optimisation_path[i,j], np.round(np.exp(theta_star[j])), demand_optimisation_path[i,j], spoil_probabilities, spoilage_optimisation_path[:,i,j], h, v, b)
            total_cost += cost*rho**j
        path_cost[i] = total_cost
    
    return np.mean(path_cost)

In [17]:
def optimise_order_decision(new_orders, number_of_optimisation_paths, init_inventory, supply_optimisation_paths, demand_optimisation_paths, spoilage_optimisation_paths, spoil_probabilities, order_paths, tau, nu, rho, h, v, b):
    inventory_start = calculate_starting_inventory(number_of_optimisation_paths, init_inventory, supply_optimisation_paths[:,:tau], order_paths, demand_optimisation_paths[:,:tau], spoil_probabilities, spoilage_optimisation_paths[:,:,:tau], tau, h, v, b)
    theta_star = np.log(new_orders)    
    res = minimize(calculate_order_costs, x0=theta_star, args=(number_of_optimisation_paths, inventory_start, supply_optimisation_paths[:,tau:], demand_optimisation_paths[:,tau:], spoil_probabilities, spoilage_optimisation_paths[:,:,tau:], nu, rho, h, v, b), method='Nelder-Mead', tol=1e-9)
        
    return res.x[0]

### Main Simulation Loop
Run the simulation over `number_of_simulation_periods`. In each period:
1. Optimization is performed to determine the order quantity.
2. The period transition is simulated (realized demand/supply).
3. Costs and inventory states are recorded.

In [18]:
def simulate_system(number_of_simulation_periods, pi, pi_stat, beta1, beta2, number_of_optimisation_paths, tau, nu, rho, type_last_period, supply_is_deterministic, init_inventory, demand_optimisation, spoil_probabilities, spoilage_optimisation, demand_evaluation, spoilage_evaluation, supply_evaluation, orders, new_orders, seed_start, h, v, b):
    total_cost = 0
    inventory_evaluation = init_inventory
    time_before = np.datetime64('now')
    cost_output = 0
    inventory_output = 0
    spoilage_output = 0
    underage_output = 0
    for t in range(number_of_simulation_periods-nu):
        cost = 0
        if t < number_of_simulation_periods-tau-nu:
            set_numpy_random_seed(t)
            seed = seed_start*t
            inventory = inventory_evaluation.copy()
            supply_optimisation = generate_supply_data_optimisation(pi, pi_stat, beta1, beta2, number_of_optimisation_paths, tau, nu, type_last_period, seed, supply_is_deterministic)
            order_append = np.round(np.exp(optimise_order_decision(new_orders, number_of_optimisation_paths, inventory, supply_optimisation, demand_optimisation[:,t:t+tau+nu+1], spoilage_optimisation[:,:,t:t+tau+nu+1], spoil_probabilities, orders[t:t+tau], tau, nu, rho, h, v, b)))
            orders = np.append(orders, order_append)
            if supply_evaluation[t] == 0:
                type_last_period = 0
            elif supply_evaluation[t] == 1:
                type_last_period = 1
            else:
                type_last_period = 2
            
        inventory_evaluation, cost, spoilage_eval, underage_eval = period_transition(inventory_evaluation, supply_evaluation[t], orders[t], demand_evaluation[t], spoil_probabilities, spoilage_evaluation[:,t], h, v, b)
        cost_output = np.append(cost_output, cost)
        inventory_output = np.append(inventory_output, np.sum(inventory_evaluation))
        spoilage_output = np.append(spoilage_output, spoilage_eval)
        underage_output = np.append(underage_output, underage_eval)
        
        if t >= tau:
            total_cost += cost
            
        if (t+nu+1) % 1000 == 0 and t>0:
            print(t)
            print(np.datetime64('now')-time_before)
    
    return orders, total_cost/(number_of_simulation_periods-tau-nu), cost_output, inventory_output, spoilage_output, underage_output


### Simulation Execution
Define simulation parameters (costs, distributions, horizons) and run the main simulation.

In [ ]:
def run_experiment(number_of_simulation_periods=5000, lambda_mu=100, lambda_omega=300, h=1, b=5, v=0.1, tau=3, nu=3, rho=0.9, demand_is_deterministic=False, supply_is_deterministic=False, spoilage_is_deterministic=False, number_of_optimisation_paths=1000, seed=678):
    pi = np.transpose(np.array([[0.99, 0.5, 0.5], [0.005, 0.4, 0.1], [0.005, 0.1, 0.4]])) # Transition probability matrix
    pi_stat = np.linalg.solve(np.transpose(np.diag(np.array([1,1,1]))-pi+1),np.array([1,1,1])) # Stationary distribution of supply
    beta1 = 2 # parameters of beta-dist
    beta2 = 3 # parameters of beta-dist

    spoil_probabilities = np.array([0.05, 0.1, 0.15, 0.35, 0.2, 0.15]) # Spoilage probabilities

    init_inventory = np.full(len(spoil_probabilities), 0, dtype=int)
    orders = np.full(tau, lambda_mu+np.sqrt(lambda_mu + lambda_omega))
    new_orders = np.full(nu+1, lambda_mu+np.sqrt(lambda_mu + lambda_omega))

    set_numpy_random_seed(seed)
    type_last_period = rand_choice_nb(np.arange(3), pi_stat)

    demand_optimisation, demand_evaluation = generate_demand_data(number_of_simulation_periods, number_of_optimisation_paths, lambda_mu, lambda_omega, seed, demand_is_deterministic)
    supply_evaluation = generate_supply_data_evaluation(number_of_simulation_periods, pi, beta1, beta2, type_last_period, seed)
    spoilage_optimisation = 0
    spoilage_optimisation, spoilage_evaluation = generate_spoilage_data(spoil_probabilities, number_of_optimisation_paths, number_of_simulation_periods, seed, spoilage_is_deterministic)

    order_result, costs_result, cost_period, inventory_period, spoilage_period, underage_period = simulate_system(number_of_simulation_periods, pi, pi_stat, beta1, beta2, number_of_optimisation_paths, tau, nu, rho, type_last_period, supply_is_deterministic, init_inventory, demand_optimisation, spoil_probabilities, spoilage_optimisation, demand_evaluation, spoilage_evaluation, supply_evaluation, orders, new_orders, seed, h, v, b)
    return order_result, costs_result, cost_period, inventory_period, spoilage_period, underage_period, demand_evaluation




In [ ]:
# Simulation Parameters
number_of_simulation_periods = 5000  # Number of periods to simulate for evaluation
lambda_mu = 100                    # Mean of the Poisson-distributed demand parameter
lambda_omega = 300                 # Variance-related parameter for demand distribution
h = 1                              # Unit spoilage cost
b = 5                              # Unit underage (out-of-stock) cost
v = 0.1                            # Unit inventory holding cost
tau = 3                            # Lead time (delay between ordering and arrival)
nu = 3                             # Planning/look-ahead horizon for optimization
rho = 0.9                          # Discount factor for future costs in optimization
demand_is_deterministic = False    # If True, demand is fixed at mean mu
supply_is_deterministic = False    # If True, supply is fixed at stationary average
spoilage_is_deterministic = False  # If True, spoilage occurs deterministically
number_of_optimisation_paths = 1000 # Number of Monte Carlo paths for optimization
seed = 678                         # Random seed for reproducibility

In [ ]:
orders, total_cost_avg, cost_period_s1, inventory_period_s1, spoilage_period_s1, underage_period_s1, demand_evaluation = run_experiment(
    number_of_simulation_periods=number_of_simulation_periods,
    lambda_mu=lambda_mu,
    lambda_omega=lambda_omega,
    h=h,
    b=b,
    v=v,
    tau=tau,
    nu=nu,
    rho=rho,
    demand_is_deterministic=demand_is_deterministic,
    supply_is_deterministic=supply_is_deterministic,
    spoilage_is_deterministic=spoilage_is_deterministic,
    number_of_optimisation_paths=number_of_optimisation_paths,
    seed=seed
)

# Alias for backward compatibility
order_result_s1 = orders
costs_result_s1 = total_cost_avg

print(f"Average Cost: {np.mean(cost_period_s1)}")

In [ ]:
# Example: Parameter Sweep for 'b' (Underage Cost)
# b_values = [1, 2, 5, 10]
# sweep_results = {}
# for b_val in b_values:
#     print(f"Running for b={b_val}...")
#     _, avg_cost, _, _, _, _, _ = run_experiment(b=b_val)
#     sweep_results[b_val] = avg_cost
# print("Sweep Results:", sweep_results)

### Results Analysis
Calculate and print average order quantity, costs, inventory levels, spoilage, and service level.

In [21]:
print(np.mean(order_result_s1[tau:]))
print(np.mean(cost_period_s1[tau:]))
print(np.mean(inventory_period_s1[tau:]))
print(np.mean(spoilage_period_s1[tau:]))
print(1-np.sum(underage_period_s1[tau:])/np.sum(demand_evaluation[tau:]))

96.32979575490589
35.542382382382385
18.93933933933934
0.9927927927927928
0.9349324153172003


### Finally, the  Newsvendor benchmark

In [22]:
#export

@njit
def period_transition_newsvendor(inventory, supply_evaluation_period, orders_period, demand_evaluation_period, spoil_probabilities, spoilage_evaluation_period, h, v, b):
    cost = 0
    inventory[0] = add_supply(supply_evaluation_period, orders_period)
    inventory, underage = take_out_demand(inventory, demand_evaluation_period)
    cost += underage*b
    inventory, spoilage = get_spoilage(spoil_probabilities, inventory, spoilage_evaluation_period)
    cost += spoilage*h + np.sum(inventory)*v
    inventory = transfer_inventory(inventory)
    
    return inventory, spoilage, underage, cost

In [ ]:
def run_newsvendor_benchmark(number_of_simulation_periods=5000, lambda_mu=100, lambda_omega=300, h=1, b=5, v=0.1, tau=3, seed=678):
    pi = np.transpose(np.array([[0.99, 0.5, 0.5], [0.005, 0.4, 0.1], [0.005, 0.1, 0.4]])) # Transition probability matrix
    pi_stat = np.linalg.solve(np.transpose(np.diag(np.array([1,1,1]))-pi+1),np.array([1,1,1])) # Stationary distribution of supply
    beta1 = 2 # parameters of beta-dist
    beta2 = 3 # parameters of beta-dist

    spoil_probabilities = np.array([0.05, 0.1, 0.15, 0.35, 0.2, 0.15]) # Spoilage probabilities

    init_inventory = np.full(len(spoil_probabilities), 0, dtype=int)
    orders = np.full(tau, lambda_mu+np.sqrt(lambda_mu + lambda_omega))

    set_numpy_random_seed(seed)
    type_last_period = rand_choice_nb(np.arange(3), pi_stat)

    demand_evaluation = np.full(number_of_simulation_periods, 0)

    for i in range(number_of_simulation_periods):
        set_numpy_random_seed(seed*(i+1))
        mu = np.random.poisson(lambda_mu)
        omega = np.random.poisson(lambda_omega)
        sigma = np.sqrt(mu + omega)

        r0 = mu*mu / (sigma*sigma - mu)
        p = (sigma*sigma - mu) / (sigma*sigma)

        set_numpy_random_seed(seed*(i+1))
        demand_evaluation[i] = np.random.negative_binomial(r0, 1-p)
        n = mu**2/(sigma**2-mu)
        p = mu/sigma**2
        orders = np.append(orders, nbinom.ppf(b/(b+h), n, p, loc=0))
        
    supply_evaluation = generate_supply_data_evaluation(number_of_simulation_periods, pi, beta1, beta2, type_last_period, seed)

    max_number_of_spoil = len(spoil_probabilities)
    set_numpy_random_seed(seed)
    spoilage_evaluation = np.array(np.zeros([max_number_of_spoil, number_of_simulation_periods]))
    for i in range(max_number_of_spoil):
        set_numpy_random_seed(seed+30*i)
        spoilage_evaluation[i,:] = np.random.uniform(0, 1, number_of_simulation_periods)

    total_cost = 0
    inventory_evaluation = init_inventory
    time_before = np.datetime64('now')
    cost_output = 0
    inventory_output = 0
    spoilage_output = 0
    underage_output = 0

    for t in range(number_of_simulation_periods):
        inventory_evaluation, spoilage_eval, underage_eval, cost_eval = period_transition_newsvendor(inventory_evaluation, supply_evaluation[t], orders[t], demand_evaluation[t], spoil_probabilities, spoilage_evaluation[:,t], h, v, b)
        cost_output = np.append(cost_output, cost_eval)
        inventory_output = np.append(inventory_output, np.sum(inventory_evaluation))
        spoilage_output = np.append(spoilage_output, spoilage_eval)
        underage_output = np.append(underage_output, underage_eval)
        
        if t >= tau:
            total_cost += cost_eval

    total_cost_avg = total_cost / (number_of_simulation_periods - tau)
    return orders, total_cost_avg, cost_output, inventory_output, spoilage_output, underage_output, demand_evaluation


In [ ]:
# Newsvendor Benchmark Parameters
nv_number_of_simulation_periods = 5000  # Number of periods to simulate for evaluation
nv_lambda_mu = 100                    # Mean of the Poisson-distributed demand parameter
nv_lambda_omega = 300                 # Variance-related parameter for demand distribution
nv_h = 1                              # Unit spoilage cost
nv_b = 5                              # Unit underage (out-of-stock) cost
nv_v = 0.1                            # Unit inventory holding cost
nv_tau = 3                            # Lead time (delay between ordering and arrival)
nv_seed = 678                         # Random seed for reproducibility


In [ ]:
nv_orders, nv_total_cost_avg, nv_cost_period, nv_inventory_period, nv_spoilage_period, nv_underage_period, nv_demand_evaluation = run_newsvendor_benchmark(
    number_of_simulation_periods=nv_number_of_simulation_periods,
    lambda_mu=nv_lambda_mu,
    lambda_omega=nv_lambda_omega,
    h=nv_h,
    b=nv_b,
    v=nv_v,
    tau=nv_tau,
    seed=nv_seed
)

print(f"Newsvendor Average Cost: {np.mean(nv_cost_period[nv_tau+1:])}")

In [ ]:
print(f"Mean Order: {np.mean(nv_orders[nv_tau:])}")
print(f"Mean Cost: {np.mean(nv_cost_period[nv_tau+1:])}")
print(f"Mean Inventory: {np.mean(nv_inventory_period[nv_tau+1:])}")
print(f"Mean Spoilage: {np.mean(nv_spoilage_period[nv_tau+1:])}")
print(f"Service Level: {1-np.sum(nv_underage_period[nv_tau+1:])/np.sum(nv_demand_evaluation[nv_tau+1:])}")